In [1]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
from typing import List, Dict
import math
import time 
load_dotenv()
openai.OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


# Chat Completion as Attendee

In [7]:
class Attendee():
    def __init__(self, model: str="gpt-3.5-turbo", temperature: float=1) -> None:
        self.client = OpenAI()
        self.model = model
        self.temperature = temperature
        self.messages = [
            {"role": "system", "content": """"
                    You are an attendee of a committee tasked with deciding the best location for the company's new office building.
                    The committee will use an active and open voting system consisting of three main components:

                    1. Decision Space: A 2D space containing the coordinates of the location options the committee must decide on. 
                    The coordinates do not imply any inherent advantages, and all options are equidistant from the origin.

                    2. Decision Mark: A marker representing the committee's collective decision. 
                    When the Decision Mark remains on an option for 3 consecutive turns, the system accepts it as the final decision, concluding the voting process.

                    3. Attendee Marks: Each attendee has a personal marker that they can freely position to influence the Decision Mark. 
                    Attendee Marks act as magnets, pulling the Decision Mark towards their preferred option. 
                    The closer the Attendee Mark is to the Decision Marker, the stronger its pull. 
                    However, you CAN NOT PLACE your marker right ON TOP the Decision Mark! LEAVE AT LEAST 0.01 UNITS from the Decision Mark.  

                    In each turn, you will receive two pieces of data:
                    - Decision Mark coordinates (DM_XY)
                    - Coordinates of every attendee's marker on previous turn [A1_XY, A2_XY, ...]

                    The voting continues until a final decision is reached, and attendees can change their opinions based on the current situation.
                    When the voting ends the attendees will be asked to describe their decision and voting process. Therefore, VOTING DECISIONS MUST BE REASONABLE. 

                    The location options are:
                    - City A: Lower cost of living, but limited access to talent pool. A_XY = [1,1]
                    - City B: Thriving tech industry, but higher competition. B_XY = [1,-1]
                    - City C: Favorable tax incentives, but less developed infrastructure. C_XY = [-1,1]
                    - City D: Proximity to key clients, but higher operational costs. D_XY = [-1,-1]

                    INSTRUCTIONS:
                    - Reply ONLY with your marker's position as two numbers between -1 and 1, separated by a comma.
                    - DO NOT include any text or explanations in your response.
                    """}
        ]
        self.current_position = [0,0]
        self.turn_result = {}
        
    def update_position_(self) -> None:
        try:
            
            # Get opininon coordinates
            completion = self.client.chat.completions.create(
                model=self.model, temperature=self.temperature, 
                messages=self.messages
            )
            current_position =  completion.choices[0].message.content.replace('[', '').replace(']', '').split(',')
            self.current_position = [float(pos) for pos in current_position]

            # Add opininon coordinates to message
            self.messages.append({"role": "assistant", "content":str(self.current_position)})
            
            DM_XY = self.turn_result['DM_XY']
            distance = math.sqrt((DM_XY[0] - self.current_position[0]) ** 2 + (DM_XY[1] - self.current_position[1]) ** 2)
            
            # Check if position on top of the decision mark
            if distance == 0:
                self.messages.append({"role": "user", "content": "You placed your marker on top of the Decision Marker. Read the instructions again and choose a new position."})
                return self.update_postion(self.turn_result)

            self.current_position
        except Exception as e:
            print(e)
    
    def inform(self,  turn_result: Dict) -> None:
        self.turn_result = turn_result
        self.messages.append({"role": "user", "content": str(self.turn_result)})
        
    def get_opinion(self) -> List:
        self.update_position_()
        return self.current_position

In [8]:
class DecisionMarker():
    
    def __init__(self) -> None:
        self.position = [0,0]
        
    def update_position(self, ATT_XY) -> None:
        """
        Calculates and updates position of the marker for given attendee marker positions
        """
        
        initial_position = self.position
        
        total_weight = 0
        final_x = 0
        final_y = 0
        
        for a_x, a_y in ATT_XY:
            distance = math.sqrt((initial_position[0] - a_x)**2 + (initial_position[1] - a_y)**2)
            if distance == 0:
                # Avoid division by zero, and truncate the effect of the relevant attendee
                continue
             
            weight = 1 / distance ** 2
            total_weight += weight 
            final_x += a_x * weight
            final_y += a_y * weight
            
        final_x /= total_weight
        final_y /= total_weight
        
        self.position = [final_x, final_y]
        
    def get_position(self) -> List:
        return self.position

In [27]:
class Moderator():
    
    def __init__(self, num_attendees: int=3) -> None:
        
        self.turn_num = 0
        self.max_turns = 10
        self.decision_marker = DecisionMarker()
        self.attendees = [Attendee() for _ in range(num_attendees)]
        self.option_xys = {'A':[1,1], 'B':[1,-1], 'C':[-1,1], 'D':[-1,-1]}
        self.turn_results = [{
            "DM_XY":[0.00, 0.00], 
            "ATT_XY":{i:[0,0] for i in range(num_attendees)}
            }]
        
    def subjective_informer_(self, att_idx):
        """
        This function exculdes the coordinate of it's own for each attendee
        """
        subjective_info = {}
        subjective_info['DM_XY'] = self.turn_results[self.turn_num]['DM_XY']
        subjective_info['ATT_XY'] = {}
        for i, _ in enumerate(self.attendees):
            if att_idx == i:
                continue
            subjective_info['ATT_XY'][i] = self.turn_results[self.turn_num]["ATT_XY"][i]
        return subjective_info
        
    def voting(self) -> None:
        
        att_positions = {}
        for i, att in enumerate(self.attendees):
            
            # Get subjective data for each attendee and feed to attendees
            info = self.subjective_informer_(i)
            att.inform(info)
            
            # Get and store attendee opinions
            att_positions[i] = att.get_opinion()
            
            
        # Calculate and get the Decison Mark's position based on given opinions
        self.decision_marker.update_position([coords for coords in att_positions.values()])
        dm_position = self.decision_marker.get_position()
        
        # Save the turn's results
        self.turn_results.append({"DM_XY": dm_position, "ATT_XY":att_positions})
        
        
    def session(self) -> None:
        for turn in range(self.max_turns):
            self.voting()
            self.turn_num = turn

In [28]:
moderator = Moderator()

In [29]:
moderator.session()

In [30]:
moderator.turn_num

9

In [31]:
moderator.turn_results

[{'DM_XY': [0.0, 0.0], 'ATT_XY': {0: [0, 0], 1: [0, 0], 2: [0, 0]}},
 {'DM_XY': [0.13071895424836602, -0.13071895424836602],
  'ATT_XY': {0: [0.8, -0.8], 1: [-0.5, -0.5], 2: [0.5, 0.5]}},
 {'DM_XY': [0.4193258748286954, 0.21342312806483382],
  'ATT_XY': {0: [0.9, -0.9], 1: [0.5, 0.5], 2: [0.3, 0.3]}},
 {'DM_XY': [0.07327137224036559, -0.07327137224036559],
  'ATT_XY': {0: [-0.2, 0.2], 1: [0.2, -0.2], 2: [0.1, -0.1]}},
 {'DM_XY': [0.4251646219214656, 0.24194103653577612],
  'ATT_XY': {0: [0.7, 0.7], 1: [0.4, 0.2], 2: [0.4, 0.2]}},
 {'DM_XY': [0.04533010797435039, -0.04533010797435039],
  'ATT_XY': {0: [0.1, -0.1], 1: [-0.1, 0.1], 2: [0.1, -0.1]}},
 {'DM_XY': [0.43577177571541414, 0.21788588785770707],
  'ATT_XY': {0: [0.6, 0.3], 1: [0.4, 0.2], 2: [0.4, 0.2]}},
 {'DM_XY': [0.03569710395026432, -0.03569710395026432],
  'ATT_XY': {0: [0.0, 0.0], 1: [0.1, -0.1], 2: [0.0, 0.0]}},
 {'DM_XY': [0.43584279652963065, 0.21792139826481532],
  'ATT_XY': {0: [0.6, 0.3], 1: [0.4, 0.2], 2: [0.4, 0.2]}}

# Assistant as Attendee

In [2]:
client = OpenAI()

In [3]:
ASSISTANT_ID = 'asst_4gAGM6MvPwgDxp1OT7qY51Yh'

In [4]:
# Create a thread with a message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "{'DM_XY': [0.13071895424836602, -0.13071895424836602], 'ATT_XY': {0: [0.8, -0.8], 1: [-0.5, -0.5], 2: [0.5, 0.5]}}"
        }
    ]
)

# Submit the thread to the assistant(as a new run)
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=ASSISTANT_ID,stream=True)

print(f"👉 Run created: {run.id}")

# wait for run to complete
while run.status != 'completed':
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"🏃‍♂️ Run Status: {run.status}")
    time.sleep(1)
else:
    print(f"🏁 Run Completed!")
    
# Get the latest message from the thread.
message_response = client.beta.threads.messages.list(thread_id=thread.id)
messages = message_response.data

# Print the latest message
latest_message = messages[0]
print(f"💬 Response: {latest_message.content[0].text.value}")

👉 Run created: run_jYJh9w8yGIDU2UL0ktEutZL1
🏃‍♂️ Run Status: in_progress
🏃‍♂️ Run Status: completed
🏁 Run Completed!
💬 Response: 0.49, -0.49


In [5]:
m = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="{'DM_XY': [0.13071895424836602, -0.13071895424836602], 'ATT_XY': {0: [0.8, -0.8], 1: [-0.5, -0.5], 2: [0.5, 0.5]}}"
    )

In [11]:
run_2 = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
run_2.status

'completed'

In [9]:

# wait for run to complete
while run.status != 'completed':
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"🏃‍♂️ Run Status: {run.status}")
    time.sleep(1)
else:
    print(f"🏁 Run Completed!")
    
# Get the latest message from the thread.
message_response = client.beta.threads.messages.list(thread_id=thread.id)
messages = message_response.data

# Print the latest message
latest_message = messages[0]
print(f"💬 Response: {latest_message.content[0].text.value}")

🏁 Run Completed!
💬 Response: {'DM_XY': [0.13071895424836602, -0.13071895424836602], 'ATT_XY': {0: [0.8, -0.8], 1: [-0.5, -0.5], 2: [0.5, 0.5]}}


In [25]:
client.beta.threads.messages.list(thread_id=thread.id).data

[ThreadMessage(id='msg_8p1WJG7NTJBjJYuBD9eqlqYe', assistant_id='asst_4gAGM6MvPwgDxp1OT7qY51Yh', content=[MessageContentText(text=Text(annotations=[], value='0.479, -0.479'), type='text')], created_at=1711369519, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_mwlY5LDrIucy0dpyPo3DiUkj', thread_id='thread_jlDPt2y2GOh3dInep9kZ3G54'),
 ThreadMessage(id='msg_FelpVD5OJLR7WzVIICBtN3mT', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value="{'DM_XY': [0.13071895424836602, -0.13071895424836602], 'ATT_XY': {0: [0.8, -0.8], 1: [-0.5, -0.5], 2: [0.5, 0.5]}}"), type='text')], created_at=1711369491, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_jlDPt2y2GOh3dInep9kZ3G54'),
 ThreadMessage(id='msg_3SasO2RuksRHwWYn1jQraGZI', assistant_id='asst_4gAGM6MvPwgDxp1OT7qY51Yh', content=[MessageContentText(text=Text(annotations=[], value='0.729, -0.729'), type='text')], created_at=1711369120, file_ids=[], 

In [2]:
# Resource: https://github.com/rokbenko/ai-playground/blob/main/openai-tutorials/6-TUI_customer_support_chatbot_streaming/tui_customer_support_chatbot_streaming.py
import os
from dotenv import load_dotenv
from openai import OpenAI, AssistantEventHandler

In [ ]:
# Resource: https://github.com/rokbenko/ai-playground/blob/main/openai-tutorials/6-TUI_customer_support_chatbot_streaming/tui_customer_support_chatbot_streaming.py
import os
from dotenv import load_dotenv
from openai import OpenAI, AssistantEventHandler
from rich.console import Console

# Load environment variables from .env
load_dotenv()

# Set up OpenAI client
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# Set up rich console for output formatting
console = Console()

# Get file and assistant IDs from environment variables
# file_id = os.getenv('OPENAI_FILE_ID')
assistant_id = os.getenv('OPENAI_ASSISTANT_ID')

# Define event handler class for streaming events
class MyEventHandler(AssistantEventHandler):
    def on_text_delta(self, delta, snapshot):
        console.print(delta.value, end = "", style = "black on white")

    def on_error(error):
        print(error)

# Create a new thread
my_thread = client.beta.threads.create()

# Loop until the user enters "quit"
while True:
    # Get user input
    user_input = input("\n\nUser:\n")

    # Check if the user wants to quit
    if user_input.lower() == "quit":
        console.print("\nAssistant:\nHave a nice day! :wave:\n\n", style = "black on white")
        break

    # Add user message to the thread
    my_thread_message = client.beta.threads.messages.create(
        thread_id = my_thread.id,
        role = "user",
        content = user_input,
        # file_ids = [file_id]
    )

    # Create and stream a run
    with client.beta.threads.runs.create_and_stream(
        thread_id = my_thread.id,
        assistant_id = assistant_id,
        event_handler = MyEventHandler(),
    ) as stream:
        console.print("\nAssistant:", style = "black on white")
        stream.until_done()